<a href="https://colab.research.google.com/github/AnuragSingh7292/Remote-sensing-image-retrieval-deep-learning-/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Transfer  Learning on  ResNet 50**

In [ ]:
from google.colab import drive
import zipfile
import os

# Mount Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Reshape, multiply, Layer

# Load ResNet50 model without the fully connected layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all layers except the last 5 layers
for layer in base_model.layers[:-5]:
    layer.trainable = False

# Define the SEBlock (Squeeze-and-Excitation Block)
class SEBlock(Layer):
    def __init__(self, ratio=16, **kwargs):
        super(SEBlock, self).__init__(**kwargs)
        self.ratio = ratio

    def build(self, input_shape):
        filters = input_shape[-1]
        self.global_avg_pool = GlobalAveragePooling2D()
        self.fc1 = Dense(filters // self.ratio, activation='relu')
        self.fc2 = Dense(filters, activation='sigmoid')
        self.reshape = Reshape((1, 1, filters))

    def call(self, inputs, **kwargs):
        se = self.global_avg_pool(inputs)
        se = self.fc1(se)
        se = self.fc2(se)
        se = self.reshape(se)
        return multiply([inputs, se])

    def get_config(self):
        config = super(SEBlock, self).get_config()
        config.update({"ratio": self.ratio})
        return config

# Add the SEBlock to the model
x = base_model.output
x = SEBlock()(x)
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
x = Dense(256, activation="relu")(x)
x = Dense(128, activation="relu")(x)
output = Dense(21, activation="softmax")(x)  # 21 classes

# Create model
model = Model(inputs=base_model.input, outputs=output)

# Print model summary
model.summary()

# Save model structure
# Define the path in your Google Drive
model_save_path = "/content/drive/MyDrive/resnet50_se_model.h5"

# Save the model
model.save(model_save_path)

print("Model saved successfully at:", model_save_path)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 25,330,197 (96.63 MB)

 Trainable params: 2,797,205 (10.67 MB)

 Non-trainable params: 22,532,992 (85.96 MB)

Model saved successfully at: /content/drive/MyDrive/resnet50_se_model.h5


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import load_model
import zipfile
import os

# # Mount Google Drive (if using Colab)
# from google.colab import drive
# drive.mount('/content/drive')

# Define dataset paths
zip_path = "/content/drive/MyDrive/Images.zip"
extract_path = "/content/images/Images"

# Unzip dataset
if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Dataset extracted successfully!")

In [ ]:

# Image preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80% train, 20% validation
)

# Load dataset
batch_size = 32
image_size = (224, 224)

train_data = train_datagen.flow_from_directory(
    extract_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Ensure categorical (one-hot encoding)
    subset='training'
)

val_data = train_datagen.flow_from_directory(
    extract_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Ensure categorical (one-hot encoding)
    subset='validation'
)

# Load pre-defined model
model = load_model("/content/drive/MyDrive/resnet50_se_model.h5", custom_objects={'SEBlock': SEBlock})

# Check the label shape
for data_batch, labels_batch in train_data:
    print(labels_batch.shape)  # Should print (batch_size, 21)
    break


# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
model_checkpoint = ModelCheckpoint("best_resnet50_se_model.h5", monitor='val_loss', save_best_only=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# Train the model
epochs = 100
print("Training the ResNet50-SE model for", epochs, "epochs...")

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
    callbacks=[model_checkpoint, lr_scheduler],
    verbose=1
)

print("Training Complete! Best model saved as 'best_resnet50_se_model.h5'.")


Found 1680 images belonging to 21 classes.
Found 420 images belonging to 21 classes.


(32, 21)
Training the ResNet50-SE model for 100 epochs...
Epoch 1/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 539ms/step - accuracy: 0.0755 - loss: 3.0086

53/53 ━━━━━━━━━━━━━━━━━━━━ 55s 799ms/step - accuracy: 0.0760 - loss: 3.0077 - val_accuracy: 0.0905 - val_loss: 3.0168 - learning_rate: 5.0000e-05
Epoch 2/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 405ms/step - accuracy: 0.1353 - loss: 2.8035

53/53 ━━━━━━━━━━━━━━━━━━━━ 29s 541ms/step - accuracy: 0.1356 - loss: 2.8029 - val_accuracy: 0.0738 - val_loss: 2.9507 - learning_rate: 5.0000e-05
Epoch 3/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 409ms/step - accuracy: 0.2144 - loss: 2.6254

53/53 ━━━━━━━━━━━━━━━━━━━━ 41s 537ms/step - accuracy: 0.2146 - loss: 2.6246 - val_accuracy: 0.0929 - val_loss: 2.8577 - learning_rate: 5.0000e-05
Epoch 4/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 415ms/step - accuracy: 0.3007 - loss: 2.4014

...

53/53 ━━━━━━━━━━━━━━━━━━━━ 28s 531ms/step - accuracy: 0.5718 - loss: 1.3157 - val_accuracy: 0.4000 - val_loss: 1.8363 - learning_rate: 1.2500e-05
Epoch 25/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 28s 524ms/step - accuracy: 0.5732 - loss: 1.2977 - val_accuracy: 0.3905 - val_loss: 1.9159 - learning_rate: 1.2500e-05
Epoch 26/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 400ms/step - accuracy: 0.5542 - loss: 1.3348

53/53 ━━━━━━━━━━━━━━━━━━━━ 32s 608ms/step - accuracy: 0.5543 - loss: 1.3347 - val_accuracy: 0.4381 - val_loss: 1.7904 - learning_rate: 1.2500e-05
Epoch 27/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 28s 521ms/step - accuracy: 0.5998 - loss: 1.2802 - val_accuracy: 0.3976 - val_loss: 1.8916 - learning_rate: 1.2500e-05
Epoch 28/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step - accuracy: 0.5492 - loss: 1.3157

53/53 ━━━━━━━━━━━━━━━━━━━━ 28s 542ms/step - accuracy: 0.5493 - loss: 1.3158 - val_accuracy: 0.4119 - val_loss: 1.7868 - learning_rate: 1.2500e-05
Epoch 29/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 28s 525ms/step - accuracy: 0.5562 - loss: 1.3210 - val_accuracy: 0.3857 - val_loss: 1.8947 - learning_rate: 1.2500e-05
Epoch 30/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 27s 511ms/step - accuracy: 0.5790 - loss: 1.3072 - val_accuracy: 0.3786 - val_loss: 1.8726 - learning_rate: 1.2500e-05
Epoch 31/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 418ms/step - accuracy: 0.5804 - loss: 1.2820
Epoch 31: ReduceLROnPlateau reducing learning rate to 6.24999984211172e-06.
53/53 ━━━━━━━━━━━━━━━━━━━━ 28s 517ms/step - accuracy: 0.5802 - loss: 1.2828 - val_accuracy: 0.4119 - val_loss: 1.8018 - learning_rate: 1.2500e-05
Epoch 32/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 27s 514ms/step - accuracy: 0.5514 - loss: 1.3177 - val_accuracy: 0.4190 - val_loss: 1.8194 - learning_rate: 6.2500e-06
Epoch 33/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 45s 594ms/step - accuracy: 

53/53 ━━━━━━━━━━━━━━━━━━━━ 28s 532ms/step - accuracy: 0.5833 - loss: 1.2691 - val_accuracy: 0.4024 - val_loss: 1.7754 - learning_rate: 1.5625e-06
Epoch 40/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 40s 522ms/step - accuracy: 0.5912 - loss: 1.2482 - val_accuracy: 0.4310 - val_loss: 1.8312 - learning_rate: 1.5625e-06
Epoch 41/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 27s 517ms/step - accuracy: 0.5752 - loss: 1.2660 - val_accuracy: 0.3857 - val_loss: 1.8320 - learning_rate: 1.5625e-06
Epoch 42/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step - accuracy: 0.6115 - loss: 1.2215
Epoch 42: ReduceLROnPlateau reducing learning rate to 7.81249980263965e-07.
53/53 ━━━━━━━━━━━━━━━━━━━━ 28s 524ms/step - accuracy: 0.6113 - loss: 1.2223 - val_accuracy: 0.4071 - val_loss: 1.8067 - learning_rate: 1.5625e-06
Epoch 43/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 28s 520ms/step - accuracy: 0.5956 - loss: 1.2502 - val_accuracy: 0.4190 - val_loss: 1.8060 - learning_rate: 7.8125e-07
Epoch 44/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 27s 514ms/step - accuracy: 

53/53 ━━━━━━━━━━━━━━━━━━━━ 28s 535ms/step - accuracy: 0.5855 - loss: 1.2777 - val_accuracy: 0.4333 - val_loss: 1.7302 - learning_rate: 9.7656e-08
Epoch 53/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 27s 520ms/step - accuracy: 0.5869 - loss: 1.2794 - val_accuracy: 0.4000 - val_loss: 1.8314 - learning_rate: 9.7656e-08
Epoch 54/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 27s 519ms/step - accuracy: 0.5925 - loss: 1.2287 - val_accuracy: 0.4095 - val_loss: 1.7905 - learning_rate: 9.7656e-08
Epoch 55/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 390ms/step - accuracy: 0.5932 - loss: 1.2460
Epoch 55: ReduceLROnPlateau reducing learning rate to 4.882812376649781e-08.
53/53 ━━━━━━━━━━━━━━━━━━━━ 27s 510ms/step - accuracy: 0.5930 - loss: 1.2463 - val_accuracy: 0.4119 - val_loss: 1.7651 - learning_rate: 9.7656e-08
Epoch 56/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 41s 517ms/step - accuracy: 0.5766 - loss: 1.2668 - val_accuracy: 0.4333 - val_loss: 1.7908 - learning_rate: 4.8828e-08
Epoch 57/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 28s 526ms/step - accuracy:

In [ ]:
model_save_path = "/content/drive/MyDrive/best_resnet50_se_model.h5"

# Save the model
model.save(model_save_path)

**Features extraction **



In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
import os

# Load the trained model
model = load_model("/content/best_resnet50_se_model.h5", custom_objects={'SEBlock': SEBlock})

# Define the layer to extract features from (penultimate layer)
feature_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)  # Output from the penultimate layer

# Set up the ImageDataGenerator without augmentation for feature extraction
datagen = ImageDataGenerator(rescale=1./255)

# Load the dataset
batch_size = 32
image_size = (224, 224)

# Set up the data generator to load all images (no validation split this time)
data = datagen.flow_from_directory(
    extract_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode=None,  # We don't need labels for feature extraction
    shuffle=False  # Do not shuffle so that we can track the image order
)

# Extract features
features = []
for i in range(len(data)):
    batch = data[i]
    feature_batch = feature_extractor.predict(batch)  # Get features from the penultimate layer
    features.append(feature_batch)

# Combine all the features
features = np.vstack(features)  # Stack all the feature vectors

# Save the features for later use
np.save('/content/drive/MyDrive/ResNet50_Transfer_image_features_Transfer.npy', features)

print("Feature extraction complete! Saved as 'image_features.npy'.")


Found 2100 images belonging to 21 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import scipy.io

# Load the feature vectors from the .npy file
features = np.load('/content/drive/MyDrive/ResNet50_Transfer_image_features_Transfer.npy')

# Define the dictionary to store in .mat file
data_dict = {'features': features}

# Save as .mat file
scipy.io.savemat('/content/drive/MyDrive/ResNet50_Transfer_image_features_Transfer.mat', data_dict)

print("Conversion complete! Saved as 'image_features.mat'.")


Conversion complete! Saved as 'image_features.mat'.


**VLAD**

In [ ]:
import numpy as np
import scipy.io
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
import scipy.io

# Load feature vectors
mat_path = "/content/drive/MyDrive/ResNet50_Transfer_image_features_Transfer.mat"
data = scipy.io.loadmat(mat_path)
features = data['features']  # Shape: (2100, 2048)

print("Original Feature Shape:", features.shape)

# Step 1: Normalize features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Step 2: Train k-means clustering model
K = 16  # Number of clusters
kmeans = MiniBatchKMeans(n_clusters=K, batch_size=256, random_state=42, max_iter=300)
kmeans.fit(features_scaled)
print("K-means clustering trained.")

# Step 3: Compute VLAD for each feature separately
def vlad_per_feature(features, kmeans):
    """ Compute VLAD encoding per feature vector (to maintain original shape). """
    N, D = features.shape  # (2100, 2048)
    K = kmeans.n_clusters  # Number of clusters

    # Get cluster assignments for each feature
    labels = kmeans.predict(features)  # Shape: (2100,)

    # Initialize VLAD feature matrix
    vlad_features = np.zeros((N, D), dtype=np.float32)  # Match input shape

    for i in range(N):
        cluster_id = labels[i]  # Get assigned cluster
        residual = features[i] - kmeans.cluster_centers_[cluster_id]  # Compute residual vector
        vlad_features[i] = residual  # Store residuals as feature representation

    # Normalize per feature
    vlad_features = vlad_features / np.sqrt(np.sum(vlad_features**2, axis=1, keepdims=True) + 1e-9)

    return vlad_features

# Apply VLAD encoding
vlad_features = vlad_per_feature(features_scaled, kmeans)
print("VLAD Feature Shape:", vlad_features.shape)  # Expected: (2100, 2048)

# Save VLAD features
output_path = "/content/drive/MyDrive/ResNet50_Transfer_image_features_Transfer_VLAD.mat"
scipy.io.savemat(output_path, {'vlad_features': vlad_features})

print(f"VLAD features saved at: {output_path}")


Original Feature Shape: (2100, 128)
K-means clustering trained.
VLAD Feature Shape: (2100, 128)
VLAD features saved at: /content/drive/MyDrive/ResNet50_Transfer_image_features_Transfer_VLAD.mat


**ARR & ARP**

In [ ]:
import numpy as np
import os

# Load the dataset (replace the path with your file)
D = np.load("/content/drive/MyDrive/image_features.npy")

# Constants for dataset
num_images = 2100

# Initialize an empty matrix for storing the sorted results
sorted_matrix = np.zeros((num_images, num_images), dtype=int)

# Function to calculate the distance
def Distance_measurement(D, Q):
    D_signed = D.astype(np.int32)
    Q_signed = Q.astype(np.int32)

    s1 = np.abs(D_signed - Q_signed)
    s11 = np.abs(1 + D_signed + Q_signed)

    p = np.divide(s1, s11)
    s2 = np.sum(p, axis=1)

    sorted_indices = np.argsort(s2)
    return sorted_indices

# Create the matrix with sorted rows
for i in range(num_images):
    Q = D[i, :]  # Query image feature vector (starts from index 0)
    sorted_indices = Distance_measurement(D, Q)  # Get sorted indices based on distance

    # Insert the sorted indices into the matrix
    sorted_matrix[i, :] = sorted_indices

# Now sorted_matrix is a 2100x2100 matrix where each row is sorted with 0-based indexing for features.


In [ ]:
# Check the shape of the matrix
print(f"Shape of the sorted_matrix: {sorted_matrix.shape}")

# Save the matrix to Google Drive for later use
np.save('/content/drive/MyDrive/sorted_UCMerced_landuse_matrix_transfer_learning.npy', sorted_matrix)

Shape of the sorted_matrix: (2100, 2100)


In [ ]:
import numpy as np

# Constants for dataset
num_images = 2100
num_categories = 21
images_per_category = 100

# Assuming the matrix is loaded from the saved file
# sorted_matrix = np.load('/content/drive/MyDrive/sorted_UCMerced_landuse_matrix.npy')

# Function to convert to binary matrix based on categories
def makes1(matrix, num_categories, images_per_category):
    # Convert list to a 2D NumPy array for easy manipulation
    matrix = np.array(matrix)

    # Loop over each category
    for i in range(num_categories):
        lower_bound = i * images_per_category
        upper_bound = (i + 1) * images_per_category

        # For rows in the range of the current category (i), replace with 1s if they belong to the category, else 0
        matrix[lower_bound:upper_bound, :] = np.where(
            (matrix[lower_bound:upper_bound, :] >= lower_bound) &
            (matrix[lower_bound:upper_bound, :] < upper_bound), 1, 0
        )

    return matrix

# Apply the function to make the matrix binary based on the categories
binary_matrix = makes1(sorted_matrix, num_categories, images_per_category)

# Print the shape of the resulting binary matrix
print(f"Shape of the binary matrix: {binary_matrix.shape}")


Shape of the binary matrix: (2100, 2100)


In [ ]:
np.save('/content/drive/MyDrive/sorted_binary_UCMerced_landuse_binary_matrix_tansfer_learning.npy', binary_matrix)

In [ ]:
for increment in range(1,11):
    sumOfList = []
    def row_sums(matrix):
        # Calculate the sum of each row
        return np.sum(matrix, axis=1)

    # print(row_sums(matrix))
    sumOfList.append(row_sums(binary_matrix))
    # print("Row Sums List:")
    # print(sumOfList)

    # Calculate p(i|Q) for each value in P
    pOfI = []
    def findP(P,Q):
        #  Calculate the precision for each query
        pOfI.append((P/Q))

    for i in sumOfList:
        findP(i,increment)
    # print("P(Iq) = \n")
    # print(pOfI)
    # Function to compute Average Retrieval Precision (ARP)
    def ARP(DB, P):
        # Flatten the list of precision values if P is a list of lists
        P_flattened = [item for sublist in P for item in sublist]
        Psum = np.sum(P_flattened)
        return (1 / DB) * Psum   # Compute ARP

    print(f"ARP = {ARP(2100, pOfI)} for {increment}")

    # Calculate recall (R) for each value in P
    ROfI = []
    def findR(P,Q):

        # Calculate the recall for each query
        ROfI.append((P/Q))

    for i in sumOfList:
        findR(i,100)
    # print(f"R(Iq) = \n {ROfI}")

    # Function to compute Average Retrieval Recall (ARR)
    def ARR(revImageInDB,R):
        R_flattened = [item for sublist in R for item in sublist]
        Rsum = np.sum(R_flattened)
        return (1 / revImageInDB) * Rsum  # Compute ARR

    print(f"ARR = {ARR(2100, ROfI)} for {increment}")


ARP = 100.0 for 1
ARR = 1.0 for 1
ARP = 50.0 for 2
ARR = 1.0 for 2
ARP = 33.333333333333336 for 3
ARR = 1.0 for 3
ARP = 25.0 for 4
ARR = 1.0 for 4
ARP = 20.0 for 5
ARR = 1.0 for 5
ARP = 16.666666666666668 for 6
ARR = 1.0 for 6
ARP = 14.285714285714286 for 7
ARR = 1.0 for 7
ARP = 12.5 for 8
ARR = 1.0 for 8
ARP = 11.111111111111112 for 9
ARR = 1.0 for 9
ARP = 10.0 for 10
ARR = 1.0 for 10


In [ ]:
for increment in range(1,101):
    sumOfList = []
    def row_sums(matrix):
        # Calculate the sum of each row
        return np.sum(matrix, axis=1)

    # print(row_sums(matrix))
    sumOfList.append(row_sums(binary_matrix))
    # print("Row Sums List:")
    # print(sumOfList)

    # Calculate p(i|Q) for each value in P
    pOfI = []
    def findP(P,Q):
        #  Calculate the precision for each query
        pOfI.append((P/Q))

    for i in sumOfList:
        findP(i,increment)
    # print("P(Iq) = \n")
    # print(pOfI)
    # Function to compute Average Retrieval Precision (ARP)
    def ARP(DB, P):
        # Flatten the list of precision values if P is a list of lists
        P_flattened = [item for sublist in P for item in sublist]
        Psum = np.sum(P_flattened)
        return (1 / DB) * Psum   # Compute ARP

    print(f"ARP = {ARP(2100, pOfI)} for {increment}")

    # Calculate recall (R) for each value in P
    ROfI = []
    def findR(P,Q):

        # Calculate the recall for each query
        ROfI.append((P/Q))

    for i in sumOfList:
        findR(i,100)
    # print(f"R(Iq) = \n {ROfI}")

    # Function to compute Average Retrieval Recall (ARR)
    def ARR(revImageInDB,R):
        R_flattened = [item for sublist in R for item in sublist]
        Rsum = np.sum(R_flattened)
        return (1 / revImageInDB) * Rsum  # Compute ARR

    print(f"ARR = {ARR(2100, ROfI)} for {increment}")


ARP = 100.0 for 1
ARR = 1.0 for 1
ARP = 50.0 for 2
ARR = 1.0 for 2
ARP = 33.333333333333336 for 3
ARR = 1.0 for 3
ARP = 25.0 for 4
ARR = 1.0 for 4
ARP = 20.0 for 5
ARR = 1.0 for 5
ARP = 16.666666666666668 for 6
ARR = 1.0 for 6
ARP = 14.285714285714286 for 7
ARR = 1.0 for 7
ARP = 12.5 for 8
ARR = 1.0 for 8
ARP = 11.111111111111112 for 9
ARR = 1.0 for 9
ARP = 10.0 for 10
ARR = 1.0 for 10
ARP = 9.090909090909092 for 11
ARR = 1.0 for 11
ARP = 8.333333333333334 for 12
ARR = 1.0 for 12
ARP = 7.692307692307693 for 13
ARR = 1.0 for 13
ARP = 7.142857142857143 for 14
ARR = 1.0 for 14
ARP = 6.666666666666667 for 15
ARR = 1.0 for 15
ARP = 6.25 for 16
ARR = 1.0 for 16
ARP = 5.88235294117647 for 17
ARR = 1.0 for 17
ARP = 5.555555555555556 for 18
ARR = 1.0 for 18
ARP = 5.263157894736843 for 19
ARR = 1.0 for 19
ARP = 5.0 for 20
ARR = 1.0 for 20
ARP = 4.761904761904761 for 21
ARR = 1.0 for 21
ARP = 4.545454545454546 for 22
ARR = 1.0 for 22
ARP = 4.347826086956522 for 23
ARR = 1.0 for 23
ARP = 4.1666666

In [ ]:
import numpy as np
import scipy.io
from sklearn.metrics.pairwise import cosine_similarity

# Load feature vectors
features = np.load('/content/drive/MyDrive/ResNet50_UCMerced_landuse_features.npy')
print("Original Feature Shape:", features.shape)

# Constants for dataset
num_images = 2100
num_categories = 21
images_per_category = 100

# Generate ground truth labels
labels = np.repeat(np.arange(num_categories), images_per_category)

# Compute similarity matrix (cosine similarity)
similarity_matrix = cosine_similarity(features)

# Function to compute precision at rank k
def compute_precision_at_k(retrieved_labels, relevant_labels, k):
    retrieved_k = retrieved_labels[:k]
    tp = sum(1 for label in retrieved_k if label in relevant_labels)
    return tp / k

# Function to compute Average Precision (AP) for a single query
def compute_ap(retrieved_labels, relevant_labels):
    ap = 0.0
    relevant_count = 0
    for k, label in enumerate(retrieved_labels, start=1):
        if label in relevant_labels:
            relevant_count += 1
            ap += compute_precision_at_k(retrieved_labels, relevant_labels, k)
    return ap / relevant_count if relevant_count > 0 else 0

# Function to compute Mean Average Precision (MAP)
def compute_map(features, labels):
    ap_values = []
    for i in range(len(features)):
        query_label = labels[i]
        relevant_labels = set(np.where(labels == query_label)[0])  # Indices of relevant images

        # Get similarity scores for query image
        similarity_scores = similarity_matrix[i]

        # Sort retrieved indices by similarity (excluding the query itself)
        retrieved_indices = np.argsort(similarity_scores)[::-1]
        retrieved_indices = retrieved_indices[retrieved_indices != i]  # Remove self-match

        # Convert indices to corresponding labels
        retrieved_labels = labels[retrieved_indices]

        # Compute AP for the current query
        ap = compute_ap(retrieved_labels, relevant_labels)
        ap_values.append(ap)

    return np.mean(ap_values) if ap_values else 0

# Compute MAP
map_score = compute_map(features, labels)
print(f"Mean Average Precision (MAP): {map_score:.4f}")


Original Feature Shape: (2100, 2048)
Mean Average Precision (MAP): 0.0476


In [ ]:
import numpy as np
import scipy.io
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

# Load feature vectors
features = np.load('/content/drive/MyDrive/ResNet50_UCMerced_landuse_features.npy')
print("Original Feature Shape:", features.shape)  # Expected: (2100, 2048)

# Normalize feature vectors (Important for cosine similarity)
features = normalize(features, axis=1)

# Constants
num_images = 2100
num_categories = 21
images_per_category = 100

# Generate ground truth labels (Each category has 100 images)
labels = np.repeat(np.arange(num_categories), images_per_category)

# Compute similarity matrix (Cosine Similarity)
similarity_matrix = cosine_similarity(features)

# Function to compute Average Precision (AP)
def compute_ap(retrieved_labels, relevant_labels):
    """
    Compute Average Precision (AP) for a single query.
    """
    ap = 0.0
    relevant_count = 0
    for k, label in enumerate(retrieved_labels, start=1):
        if label in relevant_labels:
            relevant_count += 1
            precision_at_k = relevant_count / k
            ap += precision_at_k
    return ap / len(relevant_labels) if relevant_labels else 0

# Compute MAP
def compute_map(features, labels):
    """
    Compute Mean Average Precision (MAP) over all queries.
    """
    ap_values = []
    for i in range(len(features)):
        query_label = labels[i]
        relevant_labels = set(np.where(labels == query_label)[0])  # Indices of relevant images

        # Get similarity scores for query image
        similarity_scores = similarity_matrix[i]

        # Sort retrieved indices by similarity (higher similarity first)
        retrieved_indices = np.argsort(similarity_scores)[::-1]
        retrieved_indices = retrieved_indices[retrieved_indices != i]  # Remove self-match

        # Convert indices to corresponding labels
        retrieved_labels = labels[retrieved_indices]

        # Compute AP for this query
        ap = compute_ap(retrieved_labels, relevant_labels)
        ap_values.append(ap)

    return np.mean(ap_values) if ap_values else 0

# Compute MAP
map_score = compute_map(features, labels)
print(f"Mean Average Precision (MAP): {map_score:.4f}")


Original Feature Shape: (2100, 2048)
Mean Average Precision (MAP): 0.9995


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

# Load feature vectors
features = np.load('/content/drive/MyDrive/ResNet50_UCMerced_landuse_features.npy')
print("Original Feature Shape:", features.shape)  # Expected: (2100, 2048)

# Normalize feature vectors (Important for cosine similarity)
features = normalize(features, axis=1)

# Constants
num_images = 2100
num_categories = 21
images_per_category = 100

# Generate ground truth labels (Each category has 100 images)
labels = np.repeat(np.arange(num_categories), images_per_category)

# Compute similarity matrix (Cosine Similarity)
similarity_matrix = cosine_similarity(features)

# Function to compute Average Precision (AP)
def compute_ap(retrieved_labels, relevant_labels):
    """
    Compute Average Precision (AP) for a single query.
    """
    ap = 0.0
    relevant_count = 0
    for k, label in enumerate(retrieved_labels, start=1):
        if label == relevant_labels:
            relevant_count += 1
            precision_at_k = relevant_count / k
            ap += precision_at_k
    return ap / relevant_count if relevant_count > 0 else 0

# Compute MAP
def compute_map(features, labels):
    """
    Compute Mean Average Precision (MAP) over all queries.
    """
    ap_values = []
    for i in range(len(features)):
        query_label = labels[i]

        # Get similarity scores for query image
        similarity_scores = similarity_matrix[i]

        # Sort retrieved indices by similarity (higher similarity first)
        retrieved_indices = np.argsort(similarity_scores)[::-1]

        # Remove self-match (query image itself)
        retrieved_indices = retrieved_indices[retrieved_indices != i]

        # Convert indices to corresponding labels
        retrieved_labels = labels[retrieved_indices]

        # Compute AP for this query
        ap = compute_ap(retrieved_labels, query_label)
        ap_values.append(ap)

    return np.mean(ap_values) if ap_values else 0

# Compute MAP
map_score = compute_map(features, labels)
print(f"Mean Average Precision (MAP): {map_score:.4f}")


Original Feature Shape: (2100, 2048)
Mean Average Precision (MAP): 0.1604


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

# Load feature vectors
features = np.load('/content/drive/MyDrive/ResNet50_UCMerced_landuse_features.npy')
print("Original Feature Shape:", features.shape)  # Expected: (2100, 2048)

# Normalize feature vectors (Important for cosine similarity)
features = normalize(features, axis=1)

# Constants
num_images = 2100
num_categories = 21
images_per_category = 100

# Generate ground truth labels (Each category has 100 images)
labels = np.repeat(np.arange(num_categories), images_per_category)

# Compute similarity matrix (Cosine Similarity)
similarity_matrix = cosine_similarity(features)

# Function to compute Average Precision (AP)
def compute_ap(retrieved_labels, relevant_label):
    """
    Compute Average Precision (AP) for a single query.
    """
    ap = 0.0
    relevant_count = 0
    for k, label in enumerate(retrieved_labels, start=1):
        if label == relevant_label:
            relevant_count += 1
            precision_at_k = relevant_count / k
            ap += precision_at_k
    return ap / relevant_count if relevant_count > 0 else 0

# Compute MAP
def compute_map(features, labels):
    """
    Compute Mean Average Precision (MAP) over all queries.
    """
    ap_values = []
    for i in range(len(features)):
        query_label = labels[i]

        # Get similarity scores for query image
        similarity_scores = similarity_matrix[i]

        # Sort retrieved indices by similarity (higher similarity first)
        retrieved_indices = np.argsort(similarity_scores)[::-1]

        # Remove self-match (query image itself)
        retrieved_indices = retrieved_indices[retrieved_indices != i]

        # Convert indices to corresponding labels
        retrieved_labels = labels[retrieved_indices]

        # Compute AP for this query
        ap = compute_ap(retrieved_labels, query_label)
        ap_values.append(ap)

    return np.mean(ap_values) if ap_values else 0

# Compute MAP
map_score = compute_map(features, labels)
print(f"Mean Average Precision (MAP): {map_score:.4f}")

Original Feature Shape: (2100, 2048)
Mean Average Precision (MAP): 0.1604


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

# Load feature vectors
features = np.load('/content/drive/MyDrive/ResNet50_UCMerced_landuse_features.npy')
print("Original Feature Shape:", features.shape)  # Expected: (2100, 2048)

# Normalize feature vectors (Important for cosine similarity)
features = normalize(features, axis=1)

# Constants
num_images = 2100
num_categories = 21
images_per_category = 100

# Generate ground truth labels (Each category has 100 images)
labels = np.repeat(np.arange(num_categories), images_per_category)

# Compute similarity matrix (Cosine Similarity)
similarity_matrix = cosine_similarity(features)

# Function to compute Average Precision (AP)
def compute_ap(retrieved_labels, relevant_label):
    """
    Compute Average Precision (AP) for a single query.
    """
    ap = 0.0
    relevant_count = 0
    for k, label in enumerate(retrieved_labels, start=1):
        if label == relevant_label:  # Check if the retrieved label matches the query label
            relevant_count += 1
            precision_at_k = relevant_count / k
            ap += precision_at_k
    return ap / images_per_category  # Normalize by the total number of relevant images

# Compute MAP
def compute_map(features, labels):
    """
    Compute Mean Average Precision (MAP) over all queries.
    """
    ap_values = []
    for i in range(len(features)):
        query_label = labels[i]  # Label of the query image

        # Get similarity scores for query image
        similarity_scores = similarity_matrix[i]

        # Sort retrieved indices by similarity (higher similarity first)
        retrieved_indices = np.argsort(similarity_scores)[::-1]

        # Remove self-match (query image itself)
        retrieved_indices = retrieved_indices[retrieved_indices != i]

        # Convert indices to corresponding labels
        retrieved_labels = labels[retrieved_indices]
        # Compute AP for this query
        ap = compute_ap(retrieved_labels, query_label)
        ap_values.append(ap)

        if i < 5:
          print(f"Query {i}: Label = {query_label}")
          print(f"Top 10 Retrieved Labels: {retrieved_labels[:10]}")
          print(f"AP for Query {i}: {ap:.4f}")

    return np.mean(ap_values) if ap_values else 0

# Compute MAP
map_score = compute_map(features, labels)
print(f"Mean Average Precision (MAP): {map_score:.4f}")

Original Feature Shape: (2100, 2048)
Query 0: Label = 0
Top 10 Retrieved Labels: [15 16  2  8  7  0  2  6  9  7]
AP for Query 0: 0.0676
Query 1: Label = 0
Top 10 Retrieved Labels: [20  9  0  8  2  0  9  0  0  8]
AP for Query 1: 0.1191
Query 2: Label = 0
Top 10 Retrieved Labels: [ 0 20  8  8  2  9  2  0  2 20]
AP for Query 2: 0.1326
Query 3: Label = 0
Top 10 Retrieved Labels: [2 1 8 2 9 0 9 0 8 2]
AP for Query 3: 0.0891
Query 4: Label = 0
Top 10 Retrieved Labels: [0 1 0 8 2 0 2 0 2 2]
AP for Query 4: 0.0995
Mean Average Precision (MAP): 0.1588


NameError: name 'i' is not defined

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_categories, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels))

NameError: name 'train_images' is not defined

In [ ]:
# Compute MAP
def compute_map(features, labels):
    """
    Compute Mean Average Precision (MAP) over all queries.
    """
    ap_values = []
    for i in range(len(features)):
        query_label = labels[i]  # Label of the query image

        # Get similarity scores for query image
        similarity_scores = similarity_matrix[i]

        # Sort retrieved indices by similarity (higher similarity first)
        retrieved_indices = np.argsort(similarity_scores)[::-1]

        # Remove self-match (query image itself)
        retrieved_indices = retrieved_indices[retrieved_indices != i]

        # Convert indices to corresponding labels
        retrieved_labels = labels[retrieved_indices]

        # Compute AP for this query
        ap = compute_ap(retrieved_labels, query_label)
        ap_values.append(ap)

    return np.mean(ap_values) if ap_values else 0

# Compute MAP
map_score = compute_map(features, labels)
print(f"Mean Average Precision (MAP): {map_score:.4f}")

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import os
import zipfile

# Define dataset paths
# zip_path = "/content/drive/MyDrive/Images.zip"
extract_path = "/content/images/Images/Images"

# Unzip dataset
if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Dataset extracted successfully!")

# Define image size and batch size
image_size = (224, 224)  # ResNet50 input size
batch_size = 32

# Use ImageDataGenerator to load and preprocess images
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values to [0, 1]
    validation_split=0.2   # Split dataset into 80% training and 20% validation
)

# Load training data
train_generator = datagen.flow_from_directory(
    extract_path,  # Path to the extracted dataset
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Load validation data
val_generator = datagen.flow_from_directory(
    extract_path,  # Path to the extracted dataset
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Get the number of classes
num_classes = len(train_generator.class_indices)
print(f"Number of classes: {num_classes}")

# Define the model (ResNet50)
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze initial layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=10
)

Found 1680 images belonging to 21 classes.
Found 420 images belonging to 21 classes.
Number of classes: 21


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 34s 291ms/step - accuracy: 0.0781 - loss: 3.1486 - val_accuracy: 0.0865 - val_loss: 2.7597
Epoch 2/10
 1/52 ━━━━━━━━━━━━━━━━━━━━ 4s 81ms/step - accuracy: 0.1250 - loss: 2.5774

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - accuracy: 0.1250 - loss: 2.5774 - val_accuracy: 0.0865 - val_loss: 2.7532
Epoch 3/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 20s 135ms/step - accuracy: 0.1623 - loss: 2.5852 - val_accuracy: 0.2163 - val_loss: 2.4907
Epoch 4/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.2188 - loss: 2.2800 - val_accuracy: 0.2212 - val_loss: 2.4984
Epoch 5/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 10s 199ms/step - accuracy: 0.2458 - loss: 2.2966 - val_accuracy: 0.1683 - val_loss: 2.5022
Epoch 6/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.2812 - loss: 2.2119 - val_accuracy: 0.1899 - val_loss: 2.5030
Epoch 7/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 15s 112ms/step - accuracy: 0.3050 - loss: 2.2035 - val_accuracy: 0.1779 - val_loss: 2.3984
Epoch 8/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.3125 - loss: 2.0877 - val_accuracy: 0.1827 - val_loss: 2.3644
Epoch 9/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 9s 116ms/step - accuracy: 0.3264 - loss: 2.1042 - val_accuracy: 0.2428 - val_l

In [ ]:
model.save('/content/drive/MyDrive/NewTrainModel.h5')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the saved model
model = load_model('/content/drive/MyDrive/NewTrainModel.h5')

# Load and preprocess a new image
img_path ='/content/images/Images/Images/river/river03.tif'  # Path to the new image
img = image.load_img(img_path, target_size=(224, 224))  # Resize to match model input size
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.0  # Normalize pixel values

# Make a prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)

# Map the predicted class index to the class label
class_labels = list(train_generator.class_indices.keys())
predicted_label = class_labels[predicted_class[0]]

print(f"Predicted Label: {predicted_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Predicted Label: storagetanks


In [ ]:
# Test with multiple images from the river folder
river_images = ['river01.tif', 'river02.tif', 'river03.tif', 'river04.tif']

for img_name in river_images:
    img_path = f'/content/images/Images/Images/river/{img_name}'
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    predicted_label = class_labels[predicted_class[0]]

    print(f"Image: {img_name}, Predicted Label: {predicted_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Image: river01.tif, Predicted Label: sparseresidential
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Image: river02.tif, Predicted Label: golfcourse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Image: river03.tif, Predicted Label: storagetanks
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Image: river04.tif, Predicted Label: freeway


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

# Load feature vectors
features = np.load('/content/drive/MyDrive/ResNet50_UCMerced_landuse_features.npy')  # Shape: (2100, 2048)
print("Original Feature Shape:", features.shape)

# Normalize feature vectors for better similarity computation
features = normalize(features, axis=1)

# Constants
num_images = 2100
num_categories = 21
images_per_category = 100

# Generate ground truth labels (Each category has 100 images)
labels = np.repeat(np.arange(num_categories), images_per_category)

# Compute similarity matrix (Cosine Similarity)
similarity_matrix = cosine_similarity(features)

# Function to compute Average Precision (AP) for a single query
def compute_ap(retrieved_labels, relevant_labels):
    ap = 0.0
    relevant_count = 0
    for k, label in enumerate(retrieved_labels, start=1):
        if label in relevant_labels:
            relevant_count += 1
            precision_at_k = relevant_count / k
            ap += precision_at_k
    return ap / len(relevant_labels) if len(relevant_labels) > 0 else 0

# Compute Mean Average Precision (MAP)
def compute_map(labels, similarity_matrix):
    ap_values = []
    for i in range(len(labels)):
        query_label = labels[i]
        relevant_indices = np.where(labels == query_label)[0]  # Indices of relevant images

        # Get similarity scores for query image
        similarity_scores = similarity_matrix[i]

        # Sort retrieved indices by similarity (higher similarity first)
        retrieved_indices = np.argsort(similarity_scores)[::-1]
        retrieved_indices = np.delete(retrieved_indices, np.where(retrieved_indices == i))  # Remove self-match

        # Convert indices to corresponding labels
        retrieved_labels = labels[retrieved_indices]

        # Compute AP for this query
        ap = compute_ap(retrieved_labels, relevant_indices)
        ap_values.append(ap)

    return np.mean(ap_values) if ap_values else 0

# Compute ANMRR (Average Normalized Modified Retrieval Rank)
def compute_anmrr(labels, similarity_matrix):
    anmrr_values = []
    K = 2 * images_per_category  # Maximum rank
    G = images_per_category  # Number of relevant images per query

    for i in range(len(labels)):
        query_label = labels[i]
        relevant_indices = np.where(labels == query_label)[0]

        # Get similarity scores for query image
        similarity_scores = similarity_matrix[i]

        # Sort retrieved indices by similarity (higher similarity first)
        retrieved_indices = np.argsort(similarity_scores)[::-1]
        retrieved_indices = np.delete(retrieved_indices, np.where(retrieved_indices == i))  # Remove self-match

        # Compute rank positions of relevant images (fixing the error)
        rank_positions = np.array(
            [np.where(retrieved_indices == r)[0][0] + 1 if np.where(retrieved_indices == r)[0].size > 0 else num_images
             for r in relevant_indices]
        )
        rank_positions = np.sort(rank_positions)[:G]  # Take top G ranks

        # Compute MR (Modified Retrieval Rank)
        MR = np.mean(rank_positions) + ((K - rank_positions[-1]) / 2 if rank_positions.size > 0 else 0)

        # Compute NMRR (Normalized Modified Retrieval Rank)
        ANMRR = (MR - 0.5 * (G + 1)) / (K - 0.5 * (G + 1))
        anmrr_values.append(ANMRR)

    return np.mean(anmrr_values)

# Compute MAP
map_score = compute_map(labels, similarity_matrix)
print(f"Mean Average Precision (MAP): {map_score:.4f}")

# Compute ANMRR
anmrr_score = compute_anmrr(labels, similarity_matrix)
print(f"Average Normalized Modified Retrieval Rank (ANMRR): {anmrr_score:.4f}")


Original Feature Shape: (2100, 2048)
Mean Average Precision (MAP): 0.9995
Average Normalized Modified Retrieval Rank (ANMRR): -1.6202


Average Normalized Modified Retrieval Rank (ANMRR): -1.6202


In [ ]:
def compute_anmrr(labels, similarity_matrix):
    anmrr_values = []
    K = 2 * images_per_category  # Maximum rank
    G = images_per_category  # Number of relevant images per query

    for i in range(len(labels)):
        query_label = labels[i]
        relevant_indices = np.where(labels == query_label)[0]

        # Get similarity scores for query image
        similarity_scores = similarity_matrix[i]

        # Sort retrieved indices by similarity (higher similarity first)
        retrieved_indices = np.argsort(similarity_scores)[::-1]
        retrieved_indices = np.delete(retrieved_indices, np.where(retrieved_indices == i))  # Remove self-match

        # Compute rank positions of relevant images (fixing the error)
        rank_positions = np.array(
            [np.where(retrieved_indices == r)[0][0] + 1 if np.where(retrieved_indices == r)[0].size > 0 else num_images
             for r in relevant_indices]
        )
        rank_positions = np.sort(rank_positions)[:G]  # Take top G ranks

        # Compute MR (Modified Retrieval Rank) - Avoid negative values
        if rank_positions.size > 0:
            MR = np.mean(rank_positions) + (max(K - rank_positions[-1], 0) / 2)
        else:
            MR = K  # Worst case if no relevant images are found

        # Compute NMRR (Normalized Modified Retrieval Rank)
        denominator = max(K - 0.5 * (G + 1), 1e-6)  # Prevent division by a negative number
        ANMRR = max((MR - 0.5 * (G + 1)) / denominator, 0)  # Clamp to minimum 0
        anmrr_values.append(ANMRR)

    return np.mean(anmrr_values)
# Compute ANMRR
anmrr_score = compute_anmrr(labels, similarity_matrix)
print(f"Average Normalized Modified Retrieval Rank (ANMRR): {anmrr_score:.4f}")


Average Normalized Modified Retrieval Rank (ANMRR): 4.7343
